# Inference Tools

> This module provides a comprehensive collection of tools tailored for the analysis and inference of isomatrix anndata objects. These tools facilitate the extraction of meaningful insights from complex datasets, enabling advanced data interpretation in the context of single cell resoloution isoform studies. 


In [1]:
#| default_exp inference

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
#| export
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
import scanpy as sc

In [4]:
isoform_anndata_from_long_reads = sc.read_h5ad("/data/analysis/data_mcandrew/000-sclr-discovair/discovair_long_read_transcript_matrix_annotated.h5ad")

In [5]:
isoform_anndata_from_long_reads.X

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [3., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
#| export
import numpy as np

#| export
def get_gene_mat(adata, gene_column_name='geneId'):
    # Check if the gene_column_name exists in the DataFrame
    if gene_column_name not in adata.var.columns:
        raise ValueError(f"{gene_column_name} does not exist in the adata.var DataFrame.")

    # Create a mapping from gene IDs to column indices
    gene_to_col_map = {gene_id: np.where(adata.var[gene_column_name] == gene_id)[0] for gene_id in np.unique(adata.var[gene_column_name])}

    # Sum the isoform counts per gene using vectorized operations
    gene_data = np.zeros((adata.shape[0], len(gene_to_col_map)))
    for gene_id, col_indices in gene_to_col_map.items():
        # Ensure that col_indices is always a list for proper indexing
        col_indices = np.atleast_1d(col_indices)
        gene_data[:, list(gene_to_col_map.keys()).index(gene_id)] = np.sum(adata.X[:, col_indices], axis=1)

    # Create a new AnnData object with the gene data
    gene_adata = sc.AnnData(X=gene_data, var=pd.DataFrame(index=gene_to_col_map.keys()))
    
    # Transfer relevant metadata from isoform to gene anndata
    gene_adata.obs = adata.obs.copy()
  

    return gene_adata


In [7]:
gene_matrix = get_gene_mat(isoform_anndata_from_long_reads)

In [8]:
#| export
import numpy as np

def get_iso_prop(adata, gene_column_name):
    # Ensure column exists
    if gene_column_name not in adata.var.columns:
        raise KeyError(f"Column '{gene_column_name}' does not exist in the 'adata.var' DataFrame.")

    # Get gene IDs and data
    gene_ids = adata.var[gene_column_name].values
    data = adata.X

    # Vectorized normalization
    group_sums = np.sum(data, axis=1)
    normalized_data = data / group_sums.reshape(-1, 1)

    # Fill NaNs with zeros
    normalized_data = np.nan_to_num(normalized_data, nan=0)

    return normalized_data


In [9]:
isoform_anndata_from_long_reads.obsm["isoform_proportions"] = get_iso_prop(isoform_anndata_from_long_reads, gene_column_name='geneId')


In [10]:
#| hide
import nbdev; nbdev.nbdev_export()